**Import Library**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import sympy as sp
from PIL import Image, ImageOps
import os

**Size of Data**

In [ ]:
#Size of the image
imgdir = "archive\\s1\\1.pgm"
Face = Image.open(imgdir)
FaceArray = im.pil_to_array(Face)
Height, Width = FaceArray.shape
#Number of classes
num_class = sum(1 for entry in os.scandir("archive") if not entry.is_file())
#Number of files in a class
num_files = sum(1 for entry in os.scandir("archive\\s1") if entry.is_file())
#Total number of files
num_total = num_files*num_class

**Function**

In [ ]:
#Process the Image into a flat 1D array
def ProcessImage(img):
    Grascale = ImageOps.grayscale(img)
    Facematrix = im.pil_to_array(Grascale)
    FlatFaceArray = np.concat(Facematrix,axis=None)
    return FlatFaceArray

#Make the Data Matrix
def Datamatrix(clsdir: str, num_files: int):
    for i in range(num_files):
        imgdir = clsdir +"\\" + str(i+1) + ".pgm"
    #Import image
        Face = Image.open(imgdir)
        FlatFaceArray = ProcessImage(Face)
    #Combine each 1D array in to a matrix
        if i == 0:
            FaceMatrix = FlatFaceArray
        else:
            FaceMatrix = np.column_stack((FaceMatrix,FlatFaceArray))
    return FaceMatrix

#Make a Data Matrix for a class k
def ClassMatrix(k: int):
    clsdir = "archive\\s" + str(k)
    return Datamatrix(clsdir, num_files)
ClassMatrix(2).shape



**Make the overall Data Matrix**

In [ ]:

for i in range(num_class):
    if i == 0:
        FullMatrix = ClassMatrix(i+1)
    else:
        FullMatrix = np.append(FullMatrix, ClassMatrix(i+1),axis=1)
FullMatrix.shape

(10304, 400)

In [88]:
#The average face
MeanFace=np.mean(FullMatrix, axis=1)
ColumnMeanFace= MeanFace[:, np.newaxis]
print(MeanFace)


[85.6175 85.5775 85.925  ... 76.87   75.865  75.2275]


**Calculate the eigenvalues and eigenfaces**

In [ ]:
#Minus the mean
CenteredFullMatrix=FullMatrix  - ColumnMeanFace
#Find the Eigenface of the Data
BtB=CenteredFullMatrix.T @ CenteredFullMatrix
V, S, Vt = np.linalg.svd(BtB)
#Find the needed amount of eigenvalues
eigsum=np.sum(S)
csum=0
for i in range(S.shape[0]):
    csum +=S[i]
    if csum > 0.9*eigsum:
        e90=i
        break

#S_reduced=S[0:e90]
#S_matrix = np.zeros(np.diag(S).shape)

#S_matrix[:e90, :e90] = np.sqrt(np.diag(S_reduced))
S_matrix = np.sqrt(np.diag(S))
S_inv=np.linalg.inv(S_matrix)
U=CenteredFullMatrix @ V @ S_inv
#Reducing the eigenvalues and vector to the needed amount
U_reduced = U[:,:e90]
S_reduced= np.sqrt(np.diag(S[0:e90]))


In [97]:
#Reconstruct the Eigenface as a picture
Eigenface=U_reduced @ S_reduced 
face = Eigenface[:,3]
face = np.split(face, Height)
for i in range(len(face)):
    if i == 0:
        recface=face[0]
    else:
        recface = np.vstack((recface,face[i]))
recim = Image.fromarray(recface).convert("L")
recim.save('test.png','PNG')

In [112]:
#let G be the face that need to be recognize
# the cotribution of a eigenface to it is calculated by
#Let Omega be the contribution vector

def Facespace(vector):
    Omega = np.array([])
    for i in range(e90):
        component=U_reduced[:,i] @ vector
        Omega = np.append(Omega,[[component]] )
    return Omega
#Omega= Omega[:, np.newaxis]
#Find the threshhold for the picture to be identified as a face, let that be epsilon
epsilon=0
S = 0
for i in range(num_total):
    S+= Facespace(CenteredFullMatrix[:,i])
    avgOmega = S/len(range(num_files))

for i in range(num_total):
    e = np.linalg.norm(Facespace(CenteredFullMatrix[:,i])-avgOmega)
    if e > epsilon:
        epsilon = e
avgOmega
epsilon

np.float64(5476.506638248627)

In [113]:

#Input = ProcessImage(Image.open("32.JPG"))-MeanFace
#Input.shape

image=Image.open("9.pgm")
Omega = Facespace(ProcessImage(image)-MeanFace)
epsilon > np.linalg.norm(Facespace(CenteredFullMatrix[:,2])-avgOmega)

np.True_